In [21]:
import os
# import lxml
from lxml import etree
import pandas as pd
import numpy as np

In [22]:
import sys

In [32]:
import re

In [23]:
import fasttext

In [24]:
import os

import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

In [25]:
from sklearn.tree import DecisionTreeClassifier

In [26]:
from sklearn.preprocessing import Normalizer

In [7]:
gensim.__version__

'1.0.1'

In [27]:
cl = ['заметка','интервью','статья']

In [28]:
f = open('out.csv', 'r')

file_hash = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    filename = os.path.basename(path)
    if class_name in cl:
        file_hash[filename] = class_name


In [29]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text

In [30]:
sent = []
for tree, class_text in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    all_p = tree.xpath('//p')
    joined = '\n'.join([p.text for p in all_p if p.text is not None])
    sent.append((joined, class_text))

In [31]:
len(sent)

2676

In [14]:
only_texts = [text[0] for text in sent]
classes = [text[1] for text in sent]

In [15]:
set(classes)

{'заметка', 'интервью', 'статья'}

In [16]:
type(sent[0][0])

str

In [44]:
f = open('train.txt', 'w')
p = 0
for text in sent[:2000]:
    text_clear = re.sub('\n', ' ', text[0])
    f.write(text_clear)
    f.write(' __label__'+text[1])
    f.write('\n')
    p+=1
f.close()

In [45]:
f_2 = open('test.txt', 'w')
for text in sent[2000:2676]:
    text_clear = re.sub('\n', ' ', text[0])
    f_2.write(text_clear)
    f_2.write(' __label__'+text[1])
    f_2.write('\n')

f_2.close()

In [56]:
classifier = fasttext.supervised('train.txt', 'model', label_prefix='__label__', epoch = 120)

In [57]:
result = classifier.test('test.txt')

In [58]:
print('P@1:', result.precision)
print('R@1:', result.recall)
print('Number of examples:', result.nexamples)

P@1: 0.863905325443787
R@1: 0.863905325443787
Number of examples: 676
